# Настроить блокнот

In [ ]:
# Загрузить функции для обработки логов
%run log_analyze_func.py
# Настроить matplotlib
%matplotlib inline

# Исходные данные

In [ ]:
# папка с логами
folder = r"D:\rx_logs\other\LT"
# папка, в которую будут выгружаться обработанны логи
folder_out2 = f"{folder}\\prepared"
# дата, за которую надо анализировать логи в формате ГГГГ-ММ-ДД
date = "2023-05-19"
# хосты
hosts = ['hrpro-load-rx', 'hrpro-load-rx2', 'hrpro-load-rx3',
         'hrpro-load-rx4', 'hrpro-load-rx5', 'hrpro-load-rx6']
# Временные интервалы, на которые надо разделить логи
intervals = [("00:00", "24:00")]
#intervals = [("11:20", "12:30"), ("11:54", "12:09")]
#intervals = [("00:00", "24:00"), ("11:20", "12:30")]
#intervals = [("00:00", "24:00"), ("08:00", "12:00"), ("12:00", "16:00")]
# Шаблоны для поиска логов
logs_mask = {
    'DelayOperation': '{}*.DelayOperation.{}*.log',
    'GenericService': '{}*.GenericService.{}*.log',
    'Worker': '{}*.Worker.{}*.log',
    'WebServer': '{}*.WebServer.{}*.log',
    'WorkflowProcessService': '{}*.WorkflowProcessService.{}*.log',
    'DevelopmentStudio': '{}*.DevelopmentStudio.{}*.log',
    'IntegrationService': '{}*.IntegrationService.{}*.log',
    'Sungero.ServiceRunner': '{}*.Sungero.ServiceRunner.{}*.log',
    'StorageService': '{}*.StorageService.{}*.log',
    'JobScheduler' : '{}*.JobScheduler.{}*.log'
}

# Загрузка логов в Dataframe и предвариательная обработка

In [ ]:
# Загрузить данные в DataFrame
logs = ""
logs = load_logs(folder, date, logs_mask, folder_out2, hosts)

# Разделить логи по интервалам
for interval in intervals:
    add_interval(logs, interval[0], interval[1])

# Вывести сводку и сохранить логи разбив по интервалам
for service in logs["services"]:
    for host in logs["hosts"]:
        print(f'{host}.{service}: {len(logs[service][host]["data"])}')
        if (len(logs[service][host]["data"].index) > 0):
            for interval_code in logs["intervals"]:
                print(f'  {interval_code}: {len(logs[service][host][interval_code]["data"])}')
                dataframef2log(logs, service, host, interval_code)    

# Анализ логов

In [ ]:
span = 'Span("status": "Started", "name": "Apply tag filter", "specialFolderName": "ActiveTasks")'
f = lambda d: d[(d["message"].str.contains("Apply tag filter")
                & d["message"].str.contains("ActiveTasks")
                & d["message"].str.contains("'status': 'Ok'") 
                )]
prepare_logs(logs, f,
             "ApplyTagFilter.ActiveTasks", 
             "ActiveTasks",
             show_count=True, 
             show_duration=True,
             kind = "dot")

In [ ]:
f = lambda d: d[(d["message"].str.contains("Application started"))]
prepare_logs(logs, f,
             "ApplicationStarted", 
             "ApplicationStarted",
             show_count=True, 
             show_duration=False,
             kind = "dot")

In [ ]:
f = lambda d: d[(d["message"].str.contains("Large fetch")
                )]
prepare_logs(logs, f,
             "LargeFetch", 
             "LargeFetch",
             show_count=True, 
             show_duration=True,
            visualization_func=plot_count_log2)

In [ ]:

f = lambda d: d[(d["lg"].str.contains("QueryObject")
                )]
prepare_logs(logs, f,
             "QueryObject", 
             "QueryObject",
             show_count=True, 
             show_duration=True,
            visualization_func=plot_count_log2)

# Примеры запросов